In [59]:
import numpy as np
from numpy import array
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [365]:
escalar_minmax = MinMaxScaler()
K = 10

In [187]:
#funções
def sigmoide(z):
  """Função logística Sigmóide, y^i = o(wTxi), o(z) = 1/1 + exp(-z)"""
  return 1/(1 + np.exp(-z))


def custo_cross_entropia_binaria(y, y_pred):
  """Função Custo Cross Entropy, Regressão Logística Binária"""
  sujeira = 1.0e-18 
  return -np.mean(y*(np.log(sujeira + y_pred)) + (1 - y)*np.log(sujeira + (1 - y_pred)))


In [181]:
dataset = np.genfromtxt("./breastcancer.csv", delimiter=',', skip_header=1)
X = dataset[:, :30]
X = escalar_minmax.fit_transform(X)
y = dataset[:, -1]

Questão 1

In [330]:
epocas = 1000
alfa = 0.3
# alfa = 0.01
custos = []

def regressao_logistica(X, y, W):
  n = X.shape[0]
  for i in range(epocas):
      z = X @ W
      y_pred = sigmoide(z)
      erro = y - y_pred
      
      W = W + alfa*((X.T @ erro)/n)
      custo = custo_cross_entropia_binaria(y, y_pred)
      custos.append([i, custo])
  return W

kfold = KFold(n_splits=10, random_state=42, shuffle=True)
fold = 0
classe = []
acuracia = 0.0
acuracia_classe1 = 0.0
acuracia_classe2 = 0.0
print(f'         acurácia,            std,                acurácia classe0,     acuracia clasee1')
for train_index, test_index in kfold.split(X):
  fold += 1
  X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

  _W = np.zeros(X_train.shape[1])

  # Treino
  result_W = regressao_logistica(X_train, y_train, _W)
  
  # Teste
  z_teste = X_test @ result_W
  y_predito = sigmoide(z_teste)
  

  # obter a acurrácia
  classes_preditas = [1 if i > 0.5 else 0 for i in y_predito]
  classe1 = [i if i == 0 else None for i in classes_preditas]
  classe2 = [i if i == 1 else None for i in classes_preditas]
  acuracia += np.sum(y_test == classes_preditas)*1.00/len(y_test)
  acuracia_classe1 += np.sum(y_test == classe1)*1.00/len(y_test)
  acuracia_classe2 += np.sum(y_test == classe2)*1.00/len(y_test)
  
  print(f'Fold {fold} [{acuracia/10}, {np.std(classes_preditas, dtype=np.float64)}, {acuracia_classe1/10}, {acuracia_classe2/10}]')

  # df_historico_k_custo = pd.DataFrame(data=custos, columns=['t','c'])
  # fig, (ax1) = plt.subplots(1, 1)
  # fig.suptitle("Gradiente Descendente")
  # fig.set_figwidth(12)
  # fig.set_figheight(8)
  # ax1.plot(df_historico_k_custo['t'], df_historico_k_custo['c'])

         acurácia,            std,                acurácia classe0,     acuracia clasee1
Fold 1 [0.08070175438596491, 0.4868223482635652, 0.054385964912280704, 0.02631578947368421]
Fold 2 [0.16666666666666666, 0.4962152850431912, 0.10175438596491229, 0.0649122807017544]
Fold 3 [0.2631578947368421, 0.4772445792538752, 0.1631578947368421, 0.1]
Fold 4 [0.3614035087719298, 0.4906011036529671, 0.22280701754385968, 0.13859649122807016]
Fold 5 [0.4508771929824561, 0.4714045207910317, 0.28771929824561404, 0.16315789473684209]
Fold 6 [0.5385964912280701, 0.4868223482635652, 0.3456140350877193, 0.19298245614035087]
Fold 7 [0.631578947368421, 0.4772445792538752, 0.4087719298245614, 0.22280701754385962]
Fold 8 [0.7157894736842104, 0.4714045207910317, 0.4631578947368421, 0.25263157894736843]
Fold 9 [0.8068609022556391, 0.4883855118277623, 0.518515037593985, 0.2883458646616541]
Fold 10 [0.8943609022556391, 0.49196347549842545, 0.5720864661654136, 0.32227443609022555]


# Questão 2
https://medium.com/mlearning-ai/multiclass-logistic-regression-with-python-2ee861d5772a

In [332]:
dataset = np.genfromtxt("./vehicle.csv", delimiter=',', skip_header=1)

In [353]:
X = dataset[:, :19]
X = escalar_minmax.fit_transform(X)
y = dataset[:, -1]

In [389]:
K = 10
alfa2 = 0.1
epocas2 = 1000
custos2 = []
from sklearn.utils.extmath import softmax

def my_softmax2(z):
  exp = np.exp(z)
  return (exp / np.sum(exp))

def custo_multi_cross_entropia(y, y_predi):
    """cross_entropy_loss"""
    return -np.mean(np.sum(y * np.log(y_predi)))

def acuracia(y, y_hat):
    return np.sum(y == y_hat)/len(y)

one_hot_encoder = OneHotEncoder(sparse=False)

def treina(X, y):
  _x = np.c_[np.ones(X.shape[0]), X]
  y_encode = one_hot_encoder.fit_transform(y.reshape(-1,1))
  W = np.zeros((_x.shape[1], y_encode.shape[1]))
  
  y_predi = None
  n = X.shape[0]
  for i in range(epocas2):
    z = _x @ W
    y_predi = softmax(z)
    erro = y_encode - y_predi
    
    W = W + alfa2*((_x.T @ erro)/n)
    custo2 = custo_multi_cross_entropia(y_encode, y_predi)
    custos2.append([i, custo2])
  return W
   

def testa(X, W):
  features = np.c_[np.ones(X.shape[0]), X]
  z = features @ W
  y_predi_ = softmax(z)
  return y_predi_

In [414]:

kfold = KFold(n_splits=K, random_state=42, shuffle=True)
acuracia_ = 0.0
acuracia_classe_zero,  acuracia_classe_um, acuracia_classe_dois, acuracia_classe_tres= 0.0, 0.0, 0.0, 0.0
print(f'         acurácia,            std,                acurácia classe0,     acuracia clasee1')
for train_index, test_index in kfold.split(X):

  X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
  
  ws = treina(X_train, y_train)
  # print(f'ws{ws}')
  
  y_ch = testa(X_test, ws)
  classes_preditas = np.array([np.argmax(y)*1.0 for y in y_ch])

  # obter a acurrácia
  classe_zero= [i if i == 0 else None for i in classes_preditas]
  classe_um = [i if i == 0 else None for i in classes_preditas]
  acuracia_ += np.sum(y_test == classes_preditas)*1.00/len(y_test)
  acuracia_classe_zero += np.sum(y_test == classe_zero)*1.00/len(y_test)
  acuracia_classe_um += np.sum(y_test == classe_um)*1.00/len(y_test)

  print(f'Fold {fold} [{acuracia_/K}, {np.std(classes_preditas, dtype=np.float64)}, {acuracia_classe_zero/K}, {acuracia_classe_um/K}]')

# df_historico_multi_custo = pd.DataFrame(data=custos2, columns=['t','c'])
# fig, (ax1) = plt.subplots(1, 1)
# fig.suptitle("Custo Cross Entroty")
# fig.set_figwidth(12)
# ax1.plot(df_historico_multi_custo['t'], df_historico_multi_custo['c'])

         acurácia,            std,                acurácia classe0,     acuracia clasee1
Fold 10 [0.08823529411764705, 1.1642898841165181, 0.024705882352941178, 0.024705882352941178]
Fold 10 [0.1764705882352941, 1.2621725653709506, 0.049411764705882356, 0.049411764705882356]
Fold 10 [0.2752941176470588, 1.048313854585669, 0.07529411764705882, 0.07529411764705882]
Fold 10 [0.36705882352941177, 1.1942190626305431, 0.09529411764705882, 0.09529411764705882]
Fold 10 [0.46352941176470586, 1.1697454843981643, 0.13058823529411764, 0.13058823529411764]
Fold 10 [0.5551960784313725, 1.239182215155736, 0.16511204481792716, 0.16511204481792716]
Fold 10 [0.6468627450980392, 1.1639911011028874, 0.18773109243697478, 0.18773109243697478]
Fold 10 [0.7373389355742297, 1.1050287034164088, 0.20558823529411763, 0.20558823529411763]
Fold 10 [0.8278151260504203, 1.1842695348043701, 0.23535014005602237, 0.23535014005602237]
Fold 10 [0.9218627450980392, 1.1368891796488878, 0.25796918767507, 0.25796918767507]


# MELHOR TUTORIAL
https://towardsdatascience.com/logistic-regression-from-scratch-in-python-ec66603592e2